In [3]:
import spacy

import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

sys.path.append("../")
from utils.parse_arxiv import parse_title

from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /home/zyang37/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
file = open('../data/cnn_news/filtered_dataCNN.pickle', 'rb')
data = pickle.load(file)
file.close()

text = data['Article text'].iloc[0].strip()
text

'(CNN)Working in a factory or warehouse can mean doing the same task over and over, and that repetition can lead to chronic injury. Now, a battery-powered glove could help workers by taking some of the strain.The "Ironhand" glove strengthens the wearer\'s grip, meaning they don\'t have to use as much force to perform repetitive manual tasks. Its developer, Bioservo, says it can increase the wearer\'s hand strength by 20%. The Swedish company describes the system as a "soft exoskeleton." Exoskeletons are an external device that supports and protects the body, typically increasing strength and endurance. Most have a rigid structure, but the Ironhand is soft, like a regular glove.  Photos: The robots running our warehousesRobots are an increasingly familiar presence in warehouses. At the south-east London warehouse run by British online supermarket Ocado, 3,000 robots fulfill shopping orders. When an order is sent to the warehouse, the bots spring to life and head towards the container th

In [11]:
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])

# print()
# print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

print()
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['a factory', 'warehouse', 'the same task', 'that repetition', 'chronic injury', 'a battery-powered glove', 'workers', 'some', 'the strain', 'The "Ironhand" glove', "the wearer's grip", 'they', 'as much force', 'repetitive manual tasks', 'Its developer', 'Bioservo', 'it', "the wearer's hand strength", '20%', 'The Swedish company', 'the system', 'a "soft exoskeleton', 'Exoskeletons', 'an external device', 'that', 'the body', 'strength', 'endurance', 'a rigid structure', 'the Ironhand', 'a regular glove', 'The robots', 'our warehousesRobots', 'an increasingly familiar presence', 'warehouses', 'the south-east London warehouse', 'British online supermarket Ocado', 'an order', 'the warehouse', 'the bots', 'life', 'head', 'the container', 'they', 'Scroll', 'more robots', 'that', 'warehouses', 'Caption', '8 Photos', 'The robots', 'our warehousesIn', 'response', 'the coronavirus', 'MIT', 'Ava Robotics', 'the Greater Boston Food Bank', 'a robot', 'that', 'UV light', 'the floor', '